# EXPERIMENTING IMPLEMENTATION

22 Aug 2018

[batch norm](https://www.youtube.com/watch?v=nUUqwaxLnWs) and [for testing](https://www.youtube.com/watch?v=5qefnAek8OA).

A diagnostic analysis of network structure.

In [ ]:
from __future__ import division
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import cv2

from models.darknet import Darknet

In [ ]:
# unit test for create modules
bcks = parse_cfg("cfg/yolov3.cfg")
net = create_modules(bcks)

In [ ]:
dnet = Darknet("cfg/yolov3.cfg")
weight_file = 'checkpoints/yolov3.weights'
dnet.load_net_binary(weight_file)

In [ ]:
l = list(dnet.module_list)[0][1]

In [ ]:
batch = input_images[0]
r_ = 200
c_ = 300
print(batch[0, 0][r_:r_+3, c_:c_+4])

In [ ]:
det0 = dnet(input_images[0])

In [ ]:
# debugging snippet for loading parameters.
for m_ in dnet.module_list:
    break
    
ps = list(m_.parameters())
p1 = list(m_[1].parameters())
p2 = list(m_[2].parameters())

def f_(pl):
    print(len(pl))
    for p in pl:
        print(p.shape)        
f_(ps)
print('----')
f_(p1)
print('----')
f_(p2)

UNIT TESTS

In [ ]:
pred = torch.rand(2, 255, 10, 10)
inp_dim = 320
anchors = ((116,90),  (156,198),  (373,326))
ncls = 80

In [ ]:
tmp = predict_transform(pred, inp_dim, anchors, ncls)

In [ ]:
del y

In [ ]:
x = torch.rand(2, 3, 608, 608)
import time
t0 = time.time()
y = dnet(x)
print(time.time()-t0)

In [ ]:
loaded_ims = [cv2.imread(x) for x in imlist]

im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))
im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)

In [ ]:
im1 = cv2.imread('imgs/giraffe.jpg')
im2 = cv2.imread('imgs/eagle.jpg')
loaded_ims = [im1, im2]

In [ ]:
im_batches = list(map(prep_image, loaded_ims, [412, 412]))
im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)

In [ ]:
d1 = dnet(im_batches[0])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
def show(x):
    plt.imshow(x.detach().numpy().transpose([1, 2, 0]))

In [ ]:
def bbox_iou(box1, box2):
    """
    Returns the IoU of two bounding boxes 
    """
    #Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]
    
    #get the corrdinates of the intersection rectangle
    inter_rect_x1 =  torch.max(b1_x1, b2_x1)
    inter_rect_y1 =  torch.max(b1_y1, b2_y1)
    inter_rect_x2 =  torch.min(b1_x2, b2_x2)
    inter_rect_y2 =  torch.min(b1_y2, b2_y2)
    
    #Intersection area
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) \
               * torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)

    #Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou

def write_results(pred, confid_threshold, num_classes, nms_conf=0.4):
    conf_mask = (pred[:,:,4] > confid_threshold)
    batch_size = pred.size(0)
    batch_boxes = []
    batch_classes = []
    batch_scores = []
    for i in range(batch_size):
        p_ = pred[i]
        det_indexes = conf_mask[i].nonzero().squeeze()
        if det_indexes.numel() == 0: continue
        p_ = p_[det_indexes]
        box_corner = torch.zeros_like(p_)
        box_corner[:,0] = (p_[:,0] - p_[:,2]/2.0)
        box_corner[:,1] = (p_[:,1] - p_[:,3]/2.0)
        box_corner[:,2] = (p_[:,0] + p_[:,2]/2.0)
        box_corner[:,3] = (p_[:,1] + p_[:,3]/2.0)
        boxes = box_corner[:,:4]
        scores, clss = torch.max(p_[:,5:], dim=1)
        
        for c_ in torch.unique(clss):
            c = c_.item()
            cindex = (clss==c).nonzero().squeeze()
            nc = cindex.numel()
            for i1_ in range(nc-1):
                i1 = cindex[i1_]
                i2s = cindex[i1_+1:]
                if scores[i1]<confid_threshold: # if alread supressed?
                    continue
                # supress those with much overlapping
                ious = bbox_iou(boxes[i1].unsqueeze(dim=0), boxes[i2s])
                supress_ind = i2s[(ious > nms_conf).nonzero().squeeze()]
                scores[supress_ind] = 0
                # print(i1, ious)
                # print('\t', supress_ind)
        im_valid_det_ind = (scores > confid_threshold).nonzero().squeeze()
        # 3 tensors per image in batch
        batch_boxes.append(boxes[im_valid_det_ind])
        batch_classes.append(clss[im_valid_det_ind])
        batch_scores.append(scores[im_valid_det_ind]) 
    return batch_boxes, batch_classes, batch_scores

boxes, clss, scs = write_results(pred, 0.2, 80, nms_conf=0.4)

In [ ]:
def mark_box(x, results):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results[int(x[0])]
    cls = int(x[-1])
    color = random.choice(colors)
    label = "{0}".format(classes[cls])
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2, color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), 
                cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

import os
test_dir = 'data/test_imgs'
class_names = load_classes("data/coco.names")
image_names = [os.path.join(test_dir, n_) for n_ in os.listdir(test_dir)]
images = list(map(cv2.imread, image_names))
input_images = [prep_image(im_, 416) for im_ in images]


In [ ]:
det0 = dnet(input_images[0])

In [ ]:
det0[0].shape

In [ ]:
det0[0][:,:4][(det0[0][:,4]>0.5).nonzero().squeeze()]

In [ ]:
show(input_images[1][0])

The detection results are like 
```
tensor([[   3.8977,    6.6006,  230.1316,  216.2989],
        [   4.2886,    6.5419,  257.9221,  227.7933],
        [   4.4089,    6.5725,  256.6898,  241.6108],
        [   5.2665,    6.5663,  220.9306,  234.7696],
        [   5.5405,    6.5900,  333.2379,  219.7512],
        [   8.7730,    6.9347,  155.0774,  199.1949],
        [   9.1378,    6.9157,  146.5729,  202.6583],
        [   8.7669,    7.0903,  158.5159,  196.2266],
        [   9.1075,    7.1043,  147.2384,  196.7485],
        [  24.9396,   17.3632,   28.8222,   70.3265],
        [  25.0647,   17.4225,   27.7167,   71.3745]])
```

I think this is not normal -- most 

In [ ]:
box0

In [6]:
def mark_box(out_im, box, c, score, class_names, input_size=416):
    raw_h, raw_w = out_im.shape[:2]
    scaling_factor = min(float(input_size)/raw_h, float(input_size)/raw_w)
    box[:,[1,3]] -= (input_size - scaling_factor*raw_w)/2.0
    box[:,[0,2]] -= (input_size - scaling_factor*raw_h)/2.0
    box /= scaling_factor
    box[:,1].clamp_(0.0, raw_w)
    box[:,3].clamp_(0.0, raw_w)
    box[:,0].clamp_(0.0, raw_h)
    box[:,2].clamp_(0.0, raw_h)
    
    for b_, c_, s_ in zip(box, c, score):
        c1 = tuple(b_[:2].int())
        c2 = tuple(b_[2:4].int())
        color = (0,255,0)
        label = "{0}".format(class_names[c_.item()])
        cv2.rectangle(out_im, c1, c2, color, 1)
        t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
        c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
        cv2.rectangle(out_im, c1, c2, color, -1)
        cv2.putText(out_im, label, (c1[0], c1[1] + t_size[1] + 4), 
                    cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    
    print(box)
    return out_im

In [ ]:
box0, cls0, score0 = write_results(det0, 0.5, 80)
oim = mark_box(images[0].copy(), box0[0], cls0[0], score0[0], class_names)

In [ ]:
plt.imshow(oim[:,:,(2,1,0)])

In [ ]:
show(input_images[0][0])

In [ ]:
#t1 = torch.load(os.path.expanduser("/Users/junli/tmp/yolotut_105-convolutional.pt"))
#t2 = torch.load(os.path.expanduser("/Users/junli/tmp/yolomy_105-convolutional.pt"))
t1 = torch.load(os.path.expanduser("/Users/junli/tmp/yolotut_094-yolo.pt"))
t2 = torch.load(os.path.expanduser("/Users/junli/tmp/yolomy_094-yolo.pt"))

In [ ]:
(t1-t2).abs().max()

In [ ]:
(t1[0, :, 2:]-t2[0, :, 2:]).abs().max()

In [ ]:
t1[0][:10, :2]

In [ ]:
t2[0][:10, :2]

In [ ]:
list(dnet.module_list)[85]

In [ ]:
grid = np.arange(13)
a, b = np.meshgrid(grid, grid)

x_offset = torch.FloatTensor(a).view(-1, 1)
y_offset = torch.FloatTensor(b).view(-1, 1)

x_y_offset = torch.cat(
        (x_offset, y_offset), 1).repeat(1, num_anchors)
    x_y_offset = x_y_offset.view(-1, 2).unsqueeze(0)
    pred[:, :, :2] += x_y_offset
    print(pred.shape, x_offset.shape)

在改动和学习 Yolo Detector 过程中，我想把例子中的固定大小的输入图像改成对图像大小不做要求（Fully Convolutional 家族的网络本来都有这个潜质）。所以在“翻译” final convolution layer 输出的时候，我把图像的高传入了这个整合 tensor 的函数。

```
# in forward function
...
    def forward(self, x)
        ...
        elif mod == 'yolo':
            x = transform_prediction(..., x.shape[2], ...)
```
但是发生了很诡异的检测结果。给定输入图像有反应，但是输出形状看上去合理却毫无意义的检测框。

把原tutorial重新跑了一遍，结果是对的。所以我就
1. 检查两个程序中的输入图像（处理成Tensor以后）是不是一致 （结果：是）
    - 检查两个程序中的量是不是一致只能分别写保存这些变量的子程序，再从第三个独立的笔记本里面把保存的变量读取出来比较。
2. 每一个网络 block 的输出是不是一致 （第一层就不一致）
3. 检查第一层block （```nn.Sequential``` 对象）的三个组件分别的输出是不是一致：结果发现convolution 层完全没有问题，但是过了 batch normalisation 就不一致了。
4. 检查 bn 层的参数是否一致，结果发现两个程序的bn层的4个参数确实一致。
5. 再读了一遍 Ioffe 2015 的paper，确实推导了一下 BN 的工作原理 顺便“发现了” google brain 今年做的一个很漂亮的工作：为了证实 ResNet 的 short cut 是否工作，使用分析每层的统计特征的方式发展了一项网络初始化的方法 (Xiao et al. 2018, Dynamical Isometry and Mean Field Theory of CNNs) -- 我也“顺便”复习了一下mean-field, variational inference in exponential family。
6. 毫无头绪下放弃。过后再次思考突然想起，是不是 bn 保存的时候会“残存”训练至此的样本var和mean。发现我忘记设置自己的网络是evaluation而不是training模式了。
7. 设置好eval模式后，网上下载的程序和我自己的程序里面的网络的头40个blocks都保持完全的一致。
8. 经过第一个upsampling发现结果又不一致了，原来是upsample的插值方法不一样。修正后两个网络头80层完全一致。
9. 但是我仍然感觉有问题，因为仅仅是train/eval的区别不应该导致结果的质变。
10. 果然在yolo层之后网络的输出又不一致了。但神奇的是，只有82/94/106这三个yolo层不一致，所有的convolution层都没问题。即问题出在对预测的解释上，不是预测出了问题。
11. 研读不一致的输出发现这样的patterns
```
In [38]: (t1[0, :, 2:]-t2[0, :, 2:]).abs().max()
Out[38]: tensor(0.)
In [39]: t1[0][:10, :2]
Out[39]:
tensor([[ 11.1450,   9.9402],
        [  7.7179,   9.6228],
        [ 10.2525,   5.8548],
        [ 24.0455,  11.3283],
        [ 26.1821,  12.6868],
        [ 24.8711,   6.4923],
        [ 39.8953,   9.5904],
        [ 39.9461,   9.9147],
        [ 38.7679,   5.9747],
        [ 56.5952,  10.5661]])
In [40]:t2[0][:10, :2]
Out[40]:
tensor([[ 0.6966,  0.6213],
        [ 0.4824,  0.6014],
        [ 0.6408,  0.3659],
        [ 1.5028,  0.7080],
        [ 1.6364,  0.7929],
        [ 1.5544,  0.4058],
        [ 2.4935,  0.5994],
        [ 2.4966,  0.6197],
        [ 2.4230,  0.3734],
        [ 3.5372,  0.6604]])
```
即解释预测的位置部分出了问题。这部分代码如下
```
# Add the center offsets
    grid = np.arange(grid_size)
    
    pred[:, :, :2] += x_y_offset
    ...
    pred[:, :, :4] *= stride
```
其中 offsets 只不过是grid的节点坐标。检查无误。最终发现是stride（本层的每个检测方块的大小）出了问题：
```
stride = input_dim // pred_height
```
其中 input_dim 应该是我的原始图像大小，但是这里却正好跟我的 pred_height (上一层的输出大小）一样，stride总是1. 最终原因是

> 使用 x.shape 的时候我错误的以为这是整个forward 函数的x参数，即输入图像。但是其实是我上一层输出的结果。因为我们错误的命名了变量，中间处理的结果应该约定俗成为h或者z什么的，这里因为使用了循环，就造成图一时方便，使用 `x = block[i](x)` 的形式。

一些教训和学到的东西：
- 修正这个bug耗费了我2个工作日的大部分整块时间和一些零碎时间。总计约20小时。对于一个变量名来说，可谓一字千金。
- 幸运和不幸的是 `pred[2:4]`，即框的大小也受 stride 影响，然而它们还要乘以 `anchors`，which are inversely affected by stride, 两厢抵消了，所以检测框大小正常（否则早就怀疑到这个乘数上了，但是也会错过检查出batch normalisation和upsample的问题，以及下面学习的一些经验）。
- 能够解剖模型级别的动手能力是必要的。动手能力还欠缺一点的学习者可能在上述Step 2就卡住，从而得到 Yolo 在自己的问题上不好用的结论打退堂鼓了。
- 从我的研究得到一个直观感觉。Yolo类别的检测器设计很有趣，其输出检测的层乃是从一系列convolution流水线上取样下来输出（从而只是反馈loss，并不影响计算流水线进一步工作），这样的设计是否可以跟RL的思想结合起来，比如在前序的输出confidence比较高的时候，喂给模型更近一步的信息（e.g.，early step的检测输出可以作为“是否要求zoomed-in的高分辨率数据”的决策）（中举和安成组）。
- Exponential family and graphical model是个思想上的treasure box，应该学而时习之。Data statistics (mean/variance)的迁移及其应对理论应该可以用到我们下面的Representation的工作上（安成组）。
- Yolo的解释如果换一下，也许可以比较直接地使用在人体部位和姿态的估计上，因为解释层只是重新安排调整convolution的输出的一个分支，即使很复杂也不会特别影响下面的运算（黄韬组）



In [1]:
from __future__ import division
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import cv2
import visdom
from models.darknet import Darknet, write_results
from data import prep_image, load_classes
from torchvision.transforms import ToTensor, Lambda
viz = visdom.Visdom()

In [ ]:
dnet = Darknet("cfg/yolov3.cfg")
weight_file = 'checkpoints/yolov3.weights'
dnet.load_net_binary(weight_file)
dnet.eval()

test_dir = 'data/test_imgs'
class_names = load_classes("data/coco.names")
image_names = [os.path.join(test_dir, n_) for n_ in os.listdir(test_dir)]

In [2]:
cv_images = list(map(cv2.imread, image_names))
input_images = [prep_image(im_, 416) for im_ in cv_images]
cv_img_to_tensor_prep = ToTensor()
raw_ims = [prep(im_)[[2, 1, 0]] for im_ in cv_images]

In [3]:
i = 0
pred = dnet(input_images[i])

In [17]:
corners, clss, scores = write_results(pred, 0.5)

In [18]:
corners

[tensor([[ 256.4855,   97.7296,  373.8611,  144.6448],
         [  62.9805,  103.2917,  307.9106,  273.8806],
         [  69.6696,  173.8666,  168.1434,  327.4778]])]

In [19]:


oim = mark_box(cv_images[0].copy(), corners[0], clss[0], scores[0], class_names)
oim_ = prep(oim)[[2, 1, 0]]

tensor([[ 473.5117,   84.4239,  690.2051,  171.0365],
        [ 116.2717,   94.6924,  568.4503,  409.6256],
        [ 128.6208,  224.9844,  310.4185,  508.5743]])


In [ ]:
imwnd0 = viz.image(raw_ims[0], 
                   opts=dict(title='Random!', caption='How random.'),)

In [22]:
imwnd0 = viz.image(oim_, opts=dict(title='Random!', caption='How random.'),)

- in Liu et al., given $x_1$ in Mode-1, the constraint on the corresponding $z_1$ is that, 
    - $z_1 \sim \mathcal{N}(0, \mathbf{I}_q) $ (say, $z \in \mathbb{R}^q$)
    - $z_1$ contains sufficient information about $x_1$ so that the decoder $D_1$ can recover the original signal $D_1: z_1 \mapsto \tilde{x}_1 \simeq x_1$
    - This is a bit misled ...
    
- in eq. 2, there are constraints / penalty on the discrimition on translated/generated images and the original one $L_{GAN_1}, L_{GAN_2}$, and cycle consistency $L_{CC_1}, L_{CC_2}$ -- there is an issue in those images: if more than two modes are considered, it is difficult to construct such constraints, because they are pair-wise, and the number of loss terms would be in the order of $\mathcal{O}(k^2)$, where $k$ is the number of modes.

- so if we can impose constraint on $z$, so that the encoding from different modes are "mixed", it is possible to use the unified penalty for any multi-modality transformation.
    
    
